In [93]:
import os
import csv
import threading
import subprocess
import cv2 as cv
import numpy as np
import face_recognition
from datetime import datetime

In [94]:
dir = os.getcwd()
info, encoders, names = [], [], {'None': [True, True]}
txt_file = open(os.path.join(dir, 'log', 'database.txt')).readlines()

for row in txt_file:
    contents = []
    for content in row.split():
        contents.append(content)
    info.append(contents)

for contents in info:
    img_name = contents[0]
    name = contents[1]
    names[name] = False
    img = face_recognition.load_image_file(os.path.join(dir, 'log/images', img_name))
    encoder = face_recognition.face_encodings(img)[0]
    encoders.append(encoder)

In [95]:
name = None
done = None

In [96]:
def attendence(name):
    attendence_file = open(os.path.join(dir, 'temp/attendance.csv'), 'a', newline='')
    writer = csv.writer(attendence_file)
    current_datetime = datetime.now()
    writer.writerow([current_datetime, name])
    subprocess.run(['python', 'log/speak.py', 'Hello '+name])
    attendence_file.close()

In [97]:
def model(img):
    global name, done
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    try: encoder = face_recognition.face_encodings(img)[0]
    except: encoder = np.zeros(128)
    pred = face_recognition.compare_faces(encoders, encoder)
    name = info[pred.index(max(pred))][1] if max(pred)==1 else 'None'
    if names[name]==False: 
        names[name] = True
        done = True
        threading.Thread(target=attendence(name))
    else: name = 'None'

In [98]:
from flask import Flask, request, jsonify

In [99]:
app = Flask(__name__)
@app.route('/api', methods=['POST','GET'])
def upload():
    global name, done
    received = request
    img = None
    if received.files:
        file  = received.files['imageFile']
        nparr = np.fromstring(file.read(), np.uint8)
        img = cv.imdecode(nparr, cv.IMREAD_COLOR)
        cv.imwrite(os.path.join(dir, 'temp/img.jpg'), img)
        threading.Thread(target=model(img))
        if done==True:
            done = False
        elif done==False:
            name = 'None'
        return "[SUCCESS] Image Received", name
    else:
        return "[FAILED] Image Not Received", ''
    
app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses.
 * Running on http://192.168.104.39:5000/ (Press CTRL+C to quit)
c:\Users\HP\anaconda3\envs\env_3-6-13\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if __name__ == '__main__':
192.168.104.157 - - [04/May/2023 14:52:14] "POST /api HTTP/1.1" 0 -
192.168.104.157 - - [04/May/2023 14:52:16] "POST /api HTTP/1.1" 0 -
192.168.104.157 - - [04/May/2023 14:52:17] "POST /api HTTP/1.1" 0 -
192.168.104.157 - - [04/May/2023 14:52:17] "POST /api HTTP/1.1" 0 -
192.168.104.157 - - [04/May/2023 14:52:18] "POST /api HTTP/1.1" 0 -
192.168.104.157 - - [04/May/2023 14:52:18] "POST /api HTTP/1.1" 0 -
192.168.104.157 - - [04/May/2023 14:52:18] "POST /api HTTP/1.1" 0 -
192.168.104.157 - - [04/May/2023 14:52:18] "POST /api HTTP/1.1" 0 -
192.168.104.157 - - [04/May/2023 14:52:19] "POST /api HTTP/1.1" 0 -
192.168.104.157 - - [04/May/2023 14:52:19] "POST /

192.168.104.157 - - [04/May/2023 14:53:43] "POST /api HTTP/1.1" 400 -
192.168.104.157 - - [04/May/2023 14:53:43] "POST /api HTTP/1.1" 400 -
192.168.104.157 - - [04/May/2023 14:53:43] "POST /api HTTP/1.1" 400 -
